In [ ]:
import os
import pydicom

# DICOM dosyalarının bulunduğu klasör
dcm_folder = '/content/drive/MyDrive/Teknofest-2024/Kategori4Sag/11637'

# Çıktı dosyalarının kaydedileceği klasör
output_folder = '/content/drive/MyDrive/Tekno/denemeler'
os.makedirs(output_folder, exist_ok=True)

# DICOM dosyalarını okuyun ve etiketleyin
labels = {}  # Bu sözlük, dosya adlarını etiketlerle eşleştirmek için kullanılacaktır

# Burada etiketi DICOM dosyasının metadata'sından okuyabilirsiniz
# Örnek olarak, dosya adlarına göre etiketleme yapıyoruz (dosya adlandırma konvansiyonunuza bağlı olarak değişebilir)
for root, dirs, files in os.walk(dcm_folder):
    for file_name in files:
        if file_name.endswith('.dcm'):
            file_path = os.path.join(root, file_name)
            dicom_data = pydicom.dcmread(file_path)

            # Etiket bilgilerini metadata'dan veya dosya adından çıkarın
            # Örneğin, dosya adının "normal", "kitle" veya "kalsifikasyon" içerdiğini varsayıyoruz
            if 'normal' in file_name:
                label = 'normal'
            elif 'kitle' in file_name:
                label = 'kitle'
            elif 'kalsifikasyon' in file_name:
                label = 'kalsifikasyon'
            else:
                continue  # Etiket belirlenemeyen dosyaları atla

            labels[file_name] = label

print(f"Toplam {len(labels)} etiketli dosya bulundu.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Toplam 0 etiketli dosya bulundu.


In [ ]:
import os
import pydicom
import numpy as np
from PIL import Image

# DICOM dosyalarının bulunduğu klasör
dcm_folder = '/content/drive/MyDrive/Teknofest-2024/Kategori1/10267'

# Çıktı dosyalarının kaydedileceği klasör
output_folder = '/content/drive/MyDrive/Tekno/denemeler/10267'
os.makedirs(output_folder, exist_ok=True)

# DICOM dosyalarını oku ve PNG olarak kaydet
for root, dirs, files in os.walk(dcm_folder):
    for file_name in files:
        if file_name.endswith('.dcm'):
            file_path = os.path.join(root, file_name)
            dicom_data = pydicom.dcmread(file_path)

            # DICOM verilerini numpy array olarak al
            image_array = dicom_data.pixel_array

            # Görüntüyü normalize et ve uint8 formatına dönüştür
            image_array = image_array.astype(float)
            image_array = (np.maximum(image_array, 0) / image_array.max()) * 255.0
            image_array = np.uint8(image_array)

            # Görüntüyü PIL görüntüsüne dönüştür
            image = Image.fromarray(image_array)

            # PNG olarak kaydet
            output_file_path = os.path.join(output_folder, file_name.replace('.dcm', '.png'))
            image.save(output_file_path)

print("Tüm DICOM dosyaları başarıyla PNG formatına dönüştürüldü.")


Tüm DICOM dosyaları başarıyla PNG formatına dönüştürüldü.


In [ ]:
#Kütüphane
!pip install pydicom
!pip install pillow
!pip install tensorflow
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.0 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Veri yolunu ayarlayın
data_dir = '/content/drive/MyDrive/Tekno/denemeler'  # PNG dosyalarının bulunduğu yol
classes = ['normal', 'kitle', 'kalsifikasyon']

# ImageDataGenerator kullanarak veri ön işleme
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2  # %80 eğitim, %20 doğrulama için
)

# Eğitim verileri
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Doğrulama verileri
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Modeli tanımlayın
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # 3 sınıf: normal, kitle, kalsifikasyon
])

# Modeli derleyin
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Modeli eğitin
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10  # Eğitim sayısını artırabilirsiniz
)

# Modeli kaydedin
model.save('/content/drive/MyDrive/Tekno/11.06-güncel.h5')

# Eğitimin tarihçesini görselleştirin
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
plt.xlabel('Epoch')
plt.ylabel('Doğruluk')
plt.legend(loc='lower right')
plt.show()

plt.plot(history.history['loss'], label='Eğitim Kaybı')
plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
plt.xlabel('Epoch')
plt.ylabel('Kaybı')
plt.legend(loc='upper right')
plt.show()


Found 54 images belonging to 28 classes.
Found 0 images belonging to 28 classes.
Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node categorical_crossentropy/softmax_cross_entropy_with_logits defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <la

In [ ]:
import os

data_dir = '/content/drive/MyDrive/Tekno/denemeler'

# Ana dizin içeriğini kontrol et
print("Ana dizin içeriği:")
print(os.listdir(data_dir))

# Sınıf klasörlerinin içeriğini kontrol et
for class_name in ['normal', 'kitle', 'kalsifikasyon']:
    class_dir = os.path.join(data_dir, class_name)
    print(f"{class_name} klasörü içeriği:")
    if os.path.exists(class_dir):
        print(os.listdir(class_dir))
    else:
        print(f"{class_name} klasörü bulunamadı.")


Ana dizin içeriği:
['.ipynb_checkpoints', '11637', '12443', '11374', '12539', '12614', '12682', '12321', '12369', '12411', '12012', '12065', '12137', '11824', '11866', '11894', '11779', '11718', '11372', '11403', '11436', '11078', '11121', '11180', '10082', '10144', '10210', '10267', 'normal', 'LMLO.png', 'LCC.png', 'umarımsonolur']
normal klasörü içeriği:
['LCC.png', 'LMLO.png']
kitle klasörü içeriği:
kitle klasörü bulunamadı.
kalsifikasyon klasörü içeriği:
kalsifikasyon klasörü bulunamadı.


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Veri yolunu ayarlayın
data_dir = '/content/drive/MyDrive/Tekno/denemeler'  # PNG dosyalarının bulunduğu yol
classes = ['normal', 'kitle', 'kalsifikasyon']

# ImageDataGenerator kullanarak veri ön işleme
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2  # %80 eğitim, %20 doğrulama için
)

# Eğitim verileri
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    classes=classes  # Sınıf isimlerini belirleyin
)

# Doğrulama verileri
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    classes=classes  # Sınıf isimlerini belirleyin
)

# Örnek sayısını kontrol et
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")

# steps_per_epoch ve validation_steps değerlerini hesaplayın
steps_per_epoch = np.ceil(train_generator.samples / train_generator.batch_size)
validation_steps = np.ceil(validation_generator.samples / validation_generator.batch_size)

# steps_per_epoch ve validation_steps değerlerini kontrol et
print(f"Steps per epoch: {steps_per_epoch}")
print(f"Validation steps: {validation_steps}")

# Modeli tanımlayın
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(classes), activation='softmax')  # Sınıf sayısına göre çıktıyı ayarlayın
])

# Modeli derleyin
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Eğer örnek sayısı sıfır değilse modeli eğitin
if train_generator.samples > 0 and validation_generator.samples > 0:
    # Modeli eğitin
    history = model.fit(
        train_generator,
        steps_per_epoch=int(steps_per_epoch),
        validation_data=validation_generator,
        validation_steps=int(validation_steps),
        epochs=10  # Eğitim sayısını artırabilirsiniz
    )

    # Modeli kaydedin
    model.save('/content/drive/MyDrive/Tekno/11.06-günceltest.h5')

    # Eğitimin tarihçesini görselleştirin
    plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
    plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
    plt.xlabel('Epoch')
    plt.ylabel('Doğruluk')
    plt.legend(loc='lower right')
    plt.show()

    plt.plot(history.history['loss'], label='Eğitim Kaybı')
    plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
    plt.xlabel('Epoch')
    plt.ylabel('Kaybı')
    plt.legend(loc='upper right')
    plt.show()
else:
    print("Eğitim veya doğrulama verisi bulunamadı. Lütfen veri dizinini ve sınıf isimlerini kontrol edin.")


Found 2 images belonging to 3 classes.
Found 0 images belonging to 3 classes.
Training samples: 2
Validation samples: 0
Steps per epoch: 1.0
Validation steps: 0.0
Eğitim veya doğrulama verisi bulunamadı. Lütfen veri dizinini ve sınıf isimlerini kontrol edin.


In [ ]:
import os
import pydicom
import numpy as np
from PIL import Image
import tensorflow as tf

# DICOM dosyalarının bulunduğu klasör
dcm_folder = '/content/drive/MyDrive/Tekno/10001'

# Çıktı dosyalarının kaydedileceği klasör
output_folder = '/content/drive/MyDrive/Tekno'
os.makedirs(output_folder, exist_ok=True)

# Modeli yükleyin (bu modeli kendiniz eğitmeniz veya önceden eğitilmiş bir model kullanmanız gerekebilir)
model_path = '/content/drive/My Drive/path_to_model/my_model.h5'
model = tf.keras.models.load_model(model_path)

# Etiketler ve dosya adlarını saklayacak bir sözlük
labels = {}

# DICOM dosyalarını oku ve tahmin et
for root, dirs, files in os.walk(dcm_folder):
    for file_name in files:
        if file_name.endswith('.dcm'):
            file_path = os.path.join(root, file_name)
            dicom_data = pydicom.dcmread(file_path)

            # DICOM verilerini numpy array olarak al
            image_array = dicom_data.pixel_array

            # Görüntüyü normalize et ve yeniden boyutlandır
            image_array = image_array.astype(float)
            image_array = (np.maximum(image_array, 0) / image_array.max()) * 255.0
            image_array = np.uint8(image_array)
            input_image = cv2.resize(image_array, (224, 224))
            input_image = np.expand_dims(input_image, axis=0)
            input_image = np.expand_dims(input_image, axis=-1)  # Eğer model RGB bekliyorsa, ekseni çıkarın

            # Modeli kullanarak tahmin yap
            prediction = model.predict(input_image)
            label_index = np.argmax(prediction, axis=1)[0]

            # Etiketleri belirle
            if label_index == 0:
                label = 'normal'
            elif label_index == 1:
                label = 'kitle'
            elif label_index == 2:
                label = 'kalsifikasyon'
            else:
                label = 'unknown'

            labels[file_name] = label

            # PNG olarak kaydet
            label_folder = os.path.join(output_folder, label)
            os.makedirs(label_folder, exist_ok=True)
            output_file_path = os.path.join(label_folder, file_name.replace('.dcm', '.png'))
            Image.fromarray(image_array).save(output_file_path)

print(f"Toplam {len(labels)} dosya işlendi ve etiketlendi.")

# Etiketlenmiş dosya adlarını ve etiketlerini bir dosyaya kaydedin
with open('/content/drive/MyDrive/Tekno/labels.txt', 'w') as f:
    for file_name, label in labels.items():
        f.write(f"{file_name},{label}\n")


In [ ]:
import os
import pydicom
import cv2
import numpy as np
from PIL import Image
import tensorflow as tf

# DICOM dosyalarının bulunduğu klasör
dcm_folder = '/content/drive/MyDrive/Tekno/10001'

# Çıktı dosyalarının kaydedileceği klasör
output_folder = '/content/drive/MyDrive/Tekno'
os.makedirs(output_folder, exist_ok=True)

# Modeli yükleyin (bu modeli kendiniz eğitmeniz veya önceden eğitilmiş bir model kullanmanız gerekebilir)
model_path = '/content/drive/MyDrive/Tekno/11.06-derleme.h5'
model = tf.keras.models.load_model(model_path)

# Etiketler ve dosya adlarını saklayacak bir sözlük
labels = {}

# DICOM dosyalarını oku ve tahmin et
for root, dirs, files in os.walk(dcm_folder):
    for file_name in files:
        if file_name.endswith('.dcm'):
            file_path = os.path.join(root, file_name)
            dicom_data = pydicom.dcmread(file_path)

            # DICOM verilerini numpy array olarak al
            image_array = dicom_data.pixel_array
            image_path = "/content/drive/MyDrive/Tekno/LMLO.png"
            input_image = Image.open(image_path)

            # Görüntüyü siyah beyaz formatına dönüştür
            input_image_gray = input_image.convert("L")  # "L" modu siyah beyaz formatıdır
            # Görüntüyü RGB formatına dönüştür
            input_image_rgb = input_image_gray.convert('RGB')



            # Görüntüyü normalize et ve yeniden boyutlandır
            image_array = image_array.astype(float)
            image_array = (np.maximum(image_array, 0) / image_array.max()) * 255.0
            image_array = np.uint8(image_array)
            input_image = cv2.resize(image_array, (224, 224))
            input_image = np.expand_dims(input_image, axis=0)
            input_image = np.expand_dims(input_image, axis=-1)  # Eğer model RGB bekliyorsa, ekseni çıkarın

            # Modeli kullanarak tahmin yap
            prediction = model.predict(input_image)
            label_index = np.argmax(prediction, axis=1)[0]

            # Etiketleri belirle
            if label_index == 0:
                label = 'normal'
            elif label_index == 1:
                label = 'kitle'
            elif label_index == 2:
                label = 'kalsifikasyon'
            else:
                label = 'unknown'

            labels[file_name] = label

            # PNG olarak kaydet
            label_folder = os.path.join(output_folder, label)
            os.makedirs(label_folder, exist_ok=True)
            output_file_path = os.path.join(label_folder, file_name.replace('.dcm', '.png'))
            Image.fromarray(image_array).save(output_file_path)

print(f"Toplam {len(labels)} dosya işlendi ve etiketlendi.")

# Etiketlenmiş dosya adlarını ve etiketlerini bir dosyaya kaydedin
with open('/content/drive/MyDrive/Tekno/labels.txt', 'w') as f:
    for file_name, label in labels.items():
        f.write(f"{file_name},{label}\n")

ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2381, in predict_step
        return self(x, training=False)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/input_spec.py", line 280, in assert_input_compatibility
        raise ValueError(

    ValueErro

In [ ]:
import os
import pydicom
import numpy as np
from PIL import Image
import tensorflow as tf

# DICOM dosyalarının bulunduğu klasör
dcm_folder = '/content/drive/MyDrive/Teknofest-2024/Kategori5Sol/12471'

# Çıktı dosyalarının kaydedileceği klasör
output_folder = '/content/drive/MyDrive/Tekno'
os.makedirs(output_folder, exist_ok=True)

# Modeli yükleyin (bu modeli kendiniz eğitmeniz veya önceden eğitilmiş bir model kullanmanız gerekebilir)
model_path = '/content/drive/MyDrive/Tekno/11.06-derleme.h5'
model = tf.keras.models.load_model(model_path)

# Etiketler ve dosya adlarını saklayacak bir sözlük
labels = {}

# DICOM dosyalarını oku ve tahmin et
for root, dirs, files in os.walk(dcm_folder):
    for file_name in files:
        if file_name.endswith('.dcm'):
            file_path = os.path.join(root, file_name)
            dicom_data = pydicom.dcmread(file_path)

            # DICOM verilerini numpy array olarak al
            image_array = dicom_data.pixel_array

            # Görüntüyü normalize et ve yeniden boyutlandır
            image_array = image_array.astype(float)
            image_array = (np.maximum(image_array, 0) / image_array.max()) * 255.0
            image_array = np.uint8(image_array)

            # Görüntüyü PIL görüntüsüne dönüştür
            input_image_gray = Image.fromarray(image_array)
            input_image_rgb = input_image_gray.convert("RGB")

            # Görüntüyü modelin beklediği boyutlara yeniden boyutlandır
            input_image_resized = input_image_rgb.resize((224, 224))
            input_image_resized = np.array(input_image_resized)
            input_image_resized = np.expand_dims(input_image_resized, axis=0)  # Batch boyutu ekleyin

            # Modeli kullanarak tahmin yap
            prediction = model.predict(input_image_resized)
            label_index = np.argmax(prediction, axis=1)[0]

            # Etiketleri belirle
            if label_index == 0:
                label = 'normal'
            elif label_index == 1:
                label = 'kitle'
            elif label_index == 2:
                label = 'kalsifikasyon'
            else:
                label = 'unknown'

            labels[file_name] = label

            # PNG olarak kaydet
            label_folder = os.path.join(output_folder, label)
            os.makedirs(label_folder, exist_ok=True)
            output_file_path = os.path.join(label_folder, file_name.replace('.dcm', '.png'))
            Image.fromarray(image_array).save(output_file_path)

print(f"Toplam {len(labels)} dosya işlendi ve etiketlendi.")

# Etiketlenmiş dosya adlarını ve etiketlerini bir dosyaya kaydedin
with open('/content/drive/MyDrive/Tekno/labels.txt', 'w') as f:
    for file_name, label in labels.items():
        f.write(f"{file_name},{label}\n")

1/1 [==============================] - 0s 111ms/step
Toplam 2 dosya işlendi ve etiketlendi.


In [ ]:
import os
import pydicom
import numpy as np
from PIL import Image
import tensorflow as tf

# DICOM dosyalarının bulunduğu klasör
dcm_folder = '/content/drive/MyDrive/Teknofest-2024/Kategori5Sol/12471'

# Çıktı dosyalarının kaydedileceği klasör
output_folder = '/content/drive/MyDrive/Tekno'
os.makedirs(output_folder, exist_ok=True)

# Modeli yükleyin (bu modeli kendiniz eğitmeniz veya önceden eğitilmiş bir model kullanmanız gerekebilir)
model_path = '/content/drive/MyDrive/Tekno/11.06-derleme.h5'
model = tf.keras.models.load_model(model_path)

# Etiketler ve dosya adlarını saklayacak bir sözlük
labels = {}

# DICOM dosyalarını oku ve tahmin et
for root, dirs, files in os.walk(dcm_folder):
    for file_name in files:
        if file_name.endswith('.dcm'):
            file_path = os.path.join(root, file_name)
            dicom_data = pydicom.dcmread(file_path)

            # DICOM verilerini numpy array olarak al
            image_array = dicom_data.pixel_array

            # Görüntüyü normalize et ve yeniden boyutlandır
            image_array = image_array.astype(float)
            image_array = (np.maximum(image_array, 0) / image_array.max()) * 255.0
            image_array = np.uint8(image_array)

            # Görüntüyü PIL görüntüsüne dönüştür
            input_image_gray = Image.fromarray(image_array)
            input_image_rgb = input_image_gray.convert("RGB")

            # Görüntüyü modelin beklediği boyutlara yeniden boyutlandır
            input_image_resized = input_image_rgb.resize((224, 224))
            input_image_resized = np.array(input_image_resized)
            input_image_resized = np.expand_dims(input_image_resized, axis=0)  # Batch boyutu ekleyin

            # Modeli kullanarak tahmin yap
            prediction = model.predict(input_image_resized)
            label_index = np.argmax(prediction, axis=1)[0]

            # Etiketleri belirle
            if label_index == 0:
                label = 'normal'
            elif label_index == 1:
                label = 'kitle'
            elif label_index == 2:
                label = 'kalsifikasyon'
            else:
                label = 'unknown'

            labels[file_name] = label

            # PNG olarak kaydet
            label_folder = os.path.join(output_folder, label)
            os.makedirs(label_folder, exist_ok=True)
            output_file_path = os.path.join(label_folder, file_name.replace('.dcm', '.png'))
            Image.fromarray(image_array).save(output_file_path)

print(f"Toplam {len(labels)} dosya işlendi ve etiketlendi.")

# Etiketlenmiş dosya adlarını ve etiketlerini bir dosyaya kaydedin
with open('/content/drive/MyDrive/Tekno/labels.txt', 'w') as f:
    for file_name, label in labels.items():
        f.write(f"{file_name},{label}\n")

1/1 [==============================] - 0s 217ms/step
Toplam 2 dosya işlendi ve etiketlendi.


In [ ]:
import os
import pydicom
import numpy as np
from PIL import Image
import tensorflow as tf

# DICOM dosyalarının bulunduğu klasör
dcm_folder = '/content/drive/MyDrive/Teknofest-2024/Kategori5Sol/12471'

# Çıktı dosyalarının kaydedileceği klasör
output_folder = '/content/drive/MyDrive/Tekno'
os.makedirs(output_folder, exist_ok=True)

# Modeli yükleyin (bu modeli kendiniz eğitmeniz veya önceden eğitilmiş bir model kullanmanız gerekebilir)
model_path = '/content/drive/MyDrive/Tekno/11.06-derleme.h5'
model = tf.keras.models.load_model(model_path)

# Etiketler ve dosya adlarını saklayacak bir sözlük
labels = {}

# Görüntü ve etiketler listesi
images_with_labels = []

# DICOM dosyalarını oku ve tahmin et
for root, dirs, files in os.walk(dcm_folder):
    for file_name in files:
        if file_name.endswith('.dcm'):
            file_path = os.path.join(root, file_name)
            dicom_data = pydicom.dcmread(file_path)

            # DICOM verilerini numpy array olarak al
            image_array = dicom_data.pixel_array

            # Görüntüyü normalize et ve yeniden boyutlandır
            image_array = image_array.astype(float)
            image_array = (np.maximum(image_array, 0) / image_array.max()) * 255.0
            image_array = np.uint8(image_array)

            # Görüntüyü PIL görüntüsüne dönüştür
            input_image_gray = Image.fromarray(image_array)
            input_image_rgb = input_image_gray.convert("RGB")

            # Görüntüyü modelin beklediği boyutlara yeniden boyutlandır
            input_image_resized = input_image_rgb.resize((224, 224))
            input_image_resized = np.array(input_image_resized)
            input_image_resized = np.expand_dims(input_image_resized, axis=0)  # Batch boyutu ekleyin

            # Modeli kullanarak tahmin yap
            prediction = model.predict(input_image_resized)
            label_index = np.argmax(prediction, axis=1)[0]

            # Etiketleri belirle
            if label_index == 0:
                label = 'normal'
            elif label_index == 1:
                label = 'kitle'
            elif label_index == 2:
                label = 'kalsifikasyon'
            else:
                label = 'unknown'

            labels[file_name] = label

            # PNG olarak kaydet
            label_folder = os.path.join(output_folder, label)
            os.makedirs(label_folder, exist_ok=True)
            output_file_path = os.path.join(label_folder, file_name.replace('.dcm', '.png'))
            Image.fromarray(image_array).save(output_file_path)

            # Görüntüyü ve etiketi listeye ekle
            images_with_labels.append((output_file_path, label))

print(f"Toplam {len(labels)} dosya işlendi ve etiketlendi.")

# Etiketlenmiş dosya adlarını ve etiketlerini bir dosyaya kaydedin
with open('/content/drive/MyDrive/Tekno/labels.txt', 'w') as f:
    for file_name, label in labels.items():
        f.write(f"{file_name},{label}\n")

# Görüntüler ve etiketler listesini yazdır
for image_path, label in images_with_labels:
    print(f"Görüntü: {image_path}, Etiket: {label}")

1/1 [==============================] - 0s 95ms/step
Toplam 2 dosya işlendi ve etiketlendi.
Görüntü: /content/drive/MyDrive/Tekno/kalsifikasyon/LMLO.png, Etiket: kalsifikasyon
Görüntü: /content/drive/MyDrive/Tekno/kalsifikasyon/LCC.png, Etiket: kalsifikasyon


In [ ]:
import os
import pydicom
import numpy as np
from PIL import Image
import tensorflow as tf

# DICOM dosyalarının bulunduğu klasör
dcm_folder = '/content/drive/MyDrive/Teknofest-2024/Kategori5Sol/12471'

# Çıktı dosyalarının kaydedileceği klasör
output_folder = '/content/drive/MyDrive/Tekno'
os.makedirs(output_folder, exist_ok=True)

# Modeli yükleyin (bu modeli kendiniz eğitmeniz veya önceden eğitilmiş bir model kullanmanız gerekebilir)
model_path = '/content/drive/MyDrive/Tekno/11.06-derleme.h5'
model = tf.keras.models.load_model(model_path)

# Etiketler ve dosya adlarını saklayacak bir sözlük
labels = {}

# Görüntü ve etiketler listesi
images_with_labels = []

# DICOM dosyalarını oku ve tahmin et
for root, dirs, files in os.walk(dcm_folder):
    for file_name in files:
        if file_name.endswith('.dcm'):
            file_path = os.path.join(root, file_name)
            dicom_data = pydicom.dcmread(file_path)

            # DICOM verilerini numpy array olarak al
            image_array = dicom_data.pixel_array

            # Görüntüyü normalize et ve yeniden boyutlandır
            image_array = image_array.astype(float)
            image_array = (np.maximum(image_array, 0) / image_array.max()) * 255.0
            image_array = np.uint8(image_array)

            # Görüntüyü PIL görüntüsüne dönüştür
            input_image_gray = Image.fromarray(image_array)
            input_image_rgb = input_image_gray.convert("RGB")

            # Görüntüyü modelin beklediği boyutlara yeniden boyutlandır
            input_image_resized = input_image_rgb.resize((224, 224))
            input_image_resized = np.array(input_image_resized)
            input_image_resized = np.expand_dims(input_image_resized, axis=0)  # Batch boyutu ekleyin

            # Modeli kullanarak tahmin yap
            prediction = model.predict(input_image_resized)
            label_index = np.argmax(prediction, axis=1)[0]

            # Etiketleri belirle
            if label_index == 0:
                label = 'normal'
            elif label_index == 1:
                label = 'kitle'
            elif label_index == 2:
                label = 'kalsifikasyon'
            else:
                label = 'unknown'

            labels[file_name] = label

            # PNG olarak kaydet
            label_folder = os.path.join(output_folder, label)
            os.makedirs(label_folder, exist_ok=True)
            output_file_path = os.path.join(label_folder, file_name.replace('.dcm', '.png'))
            Image.fromarray(image_array).save(output_file_path)

            # Görüntüyü ve etiketi listeye ekle
            images_with_labels.append((output_file_path, label))

print(f"Toplam {len(labels)} dosya işlendi ve etiketlendi.")

# Etiketlenmiş dosya adlarını ve etiketlerini bir dosyaya kaydedin
with open('/content/drive/MyDrive/Tekno/labels.txt', 'w') as f:
    for file_name, label in labels.items():
        f.write(f"{file_name},{label}\n")

# Görüntüler ve etiketler listesini yazdır
for image_path, label in images_with_labels:
    print(f"Görüntü: {image_path}, Etiket: {label}")

# Görüntüler ve etiketler listesini işleyin ve etiketli görüntüleri kaydedin
for image_path, label in images_with_labels:
    image = cv2.imread(image_path)

    # Etiketin görüntüye eklenmesi
    font = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (10, 30)
    fontScale = 1
    fontColor = (48, 48, 48)
    lineType = 2

    cv2.putText(image, label,
                bottomLeftCornerOfText,
                font,
                fontScale,
                fontColor,
                lineType)

    # Etiketlenmiş görüntüyü kaydet
    output_labeled_path = os.path.join(output_folder, f"labeled_{os.path.basename(image_path)}")
    cv2.imwrite(output_labeled_path, image)

1/1 [==============================] - 0s 61ms/step
Toplam 2 dosya işlendi ve etiketlendi.
Görüntü: /content/drive/MyDrive/Tekno/kalsifikasyon/LMLO.png, Etiket: kalsifikasyon
Görüntü: /content/drive/MyDrive/Tekno/kalsifikasyon/LCC.png, Etiket: kalsifikasyon


In [ ]:
import os
import pydicom
import numpy as np
import cv2
from PIL import Image, ImageDraw, ImageFont
import tensorflow as tf

# DICOM dosyalarının bulunduğu klasör
dcm_folder = '/content/drive/MyDrive/Teknofest-2024/Kategori5Sol/12471'

# Çıktı dosyalarının kaydedileceği klasör
output_folder = '/content/drive/MyDrive/Tekno/10001'
os.makedirs(output_folder, exist_ok=True)

# Modeli yükleyin (bu modeli kendiniz eğitmeniz veya önceden eğitilmiş bir model kullanmanız gerekebilir)
model_path = '/content/drive/MyDrive/Tekno/11.06-derleme.h5'
model = tf.keras.models.load_model(model_path)

# Etiketler ve dosya adlarını saklayacak bir sözlük
labels = {}

# Görüntü ve etiketler listesi
images_with_labels = []

# DICOM dosyalarını oku ve tahmin et
for root, dirs, files in os.walk(dcm_folder):
    for file_name in files:
        if file_name.endswith('.dcm'):
            file_path = os.path.join(root, file_name)
            dicom_data = pydicom.dcmread(file_path)

            # DICOM verilerini numpy array olarak al
            image_array = dicom_data.pixel_array

            # Görüntüyü normalize et ve yeniden boyutlandır
            image_array = image_array.astype(float)
            image_array = (np.maximum(image_array, 0) / image_array.max()) * 255.0
            image_array = np.uint8(image_array)

            # Görüntüyü RGB formata dönüştür
            input_image_rgb = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)

            # Görüntüyü modelin beklediği boyutlara yeniden boyutlandır
            input_image_resized = cv2.resize(input_image_rgb, (224, 224))
            input_image_resized = np.expand_dims(input_image_resized, axis=0)  # Batch boyutu ekleyin

            # Modeli kullanarak tahmin yap
            prediction = model.predict(input_image_resized)
            label_index = np.argmax(prediction, axis=1)[0]

            # Etiketleri belirle
            if label_index == 0:
                label = 'normal'
            elif label_index == 1:
                label = 'kitle'
            elif label_index == 2:
                label = 'kalsifikasyon'
            else:
                label = 'unknown'

            labels[file_name] = label

            # PNG olarak kaydet
            label_folder = os.path.join(output_folder, label)
            os.makedirs(label_folder, exist_ok=True)
            output_file_path = os.path.join(label_folder, file_name.replace('.dcm', '.png'))
            cv2.imwrite(output_file_path, image_array)

            # Görüntüyü ve etiketi listeye ekle
            images_with_labels.append((output_file_path, label))

print(f"Toplam {len(labels)} dosya işlendi ve etiketlendi.")

# Görüntüler ve etiketler listesini işleyin ve etiketli görüntüleri kaydedin
for image_path, label in images_with_labels:
    image = cv2.imread(image_path)

    # Sınırlayıcı kutuyu çizin
    if label != 'unknown':
        font = cv2.FONT_HERSHEY_SIMPLEX
        fontScale = 1
        fontColor = (255, 255, 255)
        lineType = 2
        thickness = 2

        # Sınırlayıcı kutunun koordinatlarını alın
        contours, _ = cv2.findContours(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), thickness)
            cv2.putText(image, label, (x, y - 10), font, fontScale, fontColor, lineType)

    # Etiketlenmiş görüntüyü kaydedin
    output_file_path = os.path.join(label_folder, os.path.basename(image_path))
    cv2.imwrite(output_file_path, image)

print("Etiketlenmiş görüntüler başarıyla kaydedildi.")

1/1 [==============================] - 0s 87ms/step
Toplam 2 dosya işlendi ve etiketlendi.
Etiketlenmiş görüntüler başarıyla kaydedildi.


In [ ]:
# Çalışan KOD
import os
import pydicom
import numpy as np
import cv2
from PIL import Image, ImageDraw, ImageFont
import tensorflow as tf
import csv

labels = {}

label_coordinates = []

# DICOM dosyalarının bulunduğu klasör
dcm_folder = '/content/drive/MyDrive/Teknofest-2024/Kategori5Sag/12431'

kategori = 'Kategori5'
kategorinum = '12431'

# Çıktı dosyalarının kaydedileceği klasör
output_folder = '/content/drive/MyDrive/Tekno/denemeler/umarımsonolur/12431'
os.makedirs(output_folder, exist_ok=True)

# Modeli yükleyin (bu modeli kendiniz eğitmeniz veya önceden eğitilmiş bir model kullanmanız gerekebilir)
model_path = '/content/drive/MyDrive/Tekno/11.06-derleme.h5'
model = tf.keras.models.load_model(model_path)

# Görüntü ve etiketler listesi
images_with_labels = []

# DICOM dosyalarını oku ve tahmin et
for root, dirs, files in os.walk(dcm_folder):
    for file_name in files:
        if file_name.endswith('.dcm'):
            file_path = os.path.join(root, file_name)
            dicom_data = pydicom.dcmread(file_path)

            # DICOM verilerini numpy array olarak al
            image_array = dicom_data.pixel_array

            # Görüntüyü normalize et ve yeniden boyutlandır
            image_array = image_array.astype(float)
            image_array = (np.maximum(image_array, 0) / image_array.max()) * 255.0
            image_array = np.uint8(image_array)

            # Görüntüyü RGB formata dönüştür
            input_image_rgb = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)

            # Görüntüyü modelin beklediği boyutlara yeniden boyutlandır
            input_image_resized = cv2.resize(input_image_rgb, (224, 224))
            input_image_resized = np.expand_dims(input_image_resized, axis=0)  # Batch boyutu ekleyin

            # Modeli kullanarak tahmin yap
            prediction = model.predict(input_image_resized)
            label_index = np.argmax(prediction, axis=1)[0]

            # Etiketleri belirle
            if label_index == 0:
                label = 'Normal'
            elif label_index == 1:
                label = 'Kitle'
            elif label_index == 2:
                label = 'Kalsifikasyon'
            else:
                label = 'Bulunamadı'

            # PNG olarak kaydet
            label_folder = os.path.join(output_folder, label)
            os.makedirs(label_folder, exist_ok=True)
            output_file_path = os.path.join(label_folder, file_name.replace('.dcm', '.png'))
            Image.fromarray(image_array).save(output_file_path)

            # Görüntüyü ve etiketi listeye ekle
            images_with_labels.append((output_file_path, label))

print(f"Toplam {len(images_with_labels)} dosya işlendi ve etiketlendi.")

# Görüntüler ve etiketler listesini yazdır
for image_path, label in images_with_labels:
  print(f"Görüntü: {image_path}, Etiket: {label}")

# Görüntüler ve etiketler listesini işleyin ve etiketli görüntüleri kaydedin
for image_path, label in images_with_labels:
    image = cv2.imread(image_path)

    # Sınırlayıcı kutuyu çizin
    if label != 'Bulunamadı':
        font = cv2.FONT_HERSHEY_SIMPLEX
        fontScale = 1.50
        fontColor = (48, 48, 48)
        lineType = 2
        thickness = 2
        #print(label)

        # Sınırlayıcı kutunun koordinatlarını alın
        # Etiketin görüntüye eklenmesi
        bottomLeftCornerOfText = (10, 50)
        contours, _ = cv2.findContours(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            #cv2.rectangle(image, (x, y), (x + w, y + h), (48, 255, 48), thickness)
            cv2.putText(image, label, bottomLeftCornerOfText, font, fontScale, fontColor, lineType)
            coordinate_string = f"{x},{y};{x},{y + h};{x + w},{y + h};{x + w},{y}"
            label_coordinates.append((file_name, label, coordinate_string))

    # Etiketlenmiş görüntüyü kaydedin
    output_file_path = os.path.join(output_folder, os.path.basename(image_path))
    cv2.imwrite(output_file_path, image)

    labels[file_name] = label

    # Etiketlenmiş görüntüyü kaydet
    output_labeled_path = os.path.join(output_folder, f"labeled_{os.path.basename(image_path)}")
    cv2.imwrite(output_labeled_path, image)

    # Modeli kullanarak tahmin yap
    prediction = model.predict(input_image_resized)[0]

    item = label_coordinates
csv_file_path = '/content/drive/MyDrive/Tekno/labels.csv'
file_exists = os.path.isfile(csv_file_path)

with open(csv_file_path, 'a', newline='', encoding='utf-8') as csvfile:  # 'a' modunda açarak dosyaya ekleme yapıyoruz
    csvwriter = csv.writer(csvfile)
    if not file_exists:
        csvwriter.writerow(["KATEGORİ", "HASTA ID", "DOSYA ADI", "ETİKET ADI", "ETİKET KOORDİNATLARI"])  # Başlıkları yaz
    for item in label_coordinates:
        for file_name, label in labels.items():
          print("Konum Değeleri: " + item[2])  # Değerleri görmek için
    csvwriter.writerow([kategori, kategorinum, file_name, label, item[2]])

print(f"Toplam {len(labels)} dosya işlendi ve etiketlendi.")

1/1 [==============================] - 0s 158ms/step
Toplam 2 dosya işlendi ve etiketlendi.
Görüntü: /content/drive/MyDrive/Tekno/denemeler/umarımsonolur/12431/Kitle/RCC.png, Etiket: Kitle
Görüntü: /content/drive/MyDrive/Tekno/denemeler/umarımsonolur/12431/Kitle/RMLO.png, Etiket: Kitle
1/1 [==============================] - 0s 221ms/step
Konum Değeleri: 0,0;0,2964;2364,2964;2364,0
Konum Değeleri: 0,0;0,2964;2364,2964;2364,0
Toplam 1 dosya işlendi ve etiketlendi.


In [ ]:
!pip install pydicom
!pip install pillow
!pip install tensorflow
!pip install scikit-learn
!pip install opencv-python-headless

In [ ]:
import os
import pydicom
import numpy as np
from PIL import Image

# DICOM dosyalarının bulunduğu klasör
dcm_folder = '/content/drive/MyDrive/Tekno/10001'

# Çıktı dosyalarının kaydedileceği klasör
output_folder = '/content/drive/MyDrive/Tekno'
os.makedirs(output_folder, exist_ok=True)

# DICOM dosyalarını PNG formatına dönüştür ve etiketli klasörlere kaydet
for root, dirs, files in os.walk(dcm_folder):
    for file_name in files:
        if file_name.endswith('.dcm'):
            file_path = os.path.join(root, file_name)
            dicom_data = pydicom.dcmread(file_path)

            # DICOM verilerini numpy array olarak al
            image_array = dicom_data.pixel_array

            # Görüntüyü normalize et (isteğe bağlı)
            image_array = image_array.astype(float)
            image_array = (np.maximum(image_array, 0) / image_array.max()) * 255.0
            image_array = np.uint8(image_array)

            # PIL kullanarak görüntü oluştur
            image = Image.fromarray(image_array)

            # Etikete göre klasör oluştur ve PNG olarak kaydet
            if 'normal' in file_name.lower():
                label = 'normal'
            elif 'kitle' in file_name.lower():
                label = 'kitle'
            elif 'kalsifikasyon' in file_name.lower():
                label = 'kalsifikasyon'
            else:
                label = 'unknown'  # Tanımlanamayan dosyalar

            label_folder = os.path.join(output_folder, label)
            os.makedirs(label_folder, exist_ok=True)
            output_file_path = os.path.join(label_folder, file_name.replace('.dcm', '.png'))
            image.save(output_file_path)

print("Tüm dosyalar başarıyla PNG formatına dönüştürüldü.")

Tüm dosyalar başarıyla PNG formatına dönüştürüldü.


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Veri yolunu ayarlayın
data_dir = output_folder  # PNG dosyalarının bulunduğu yol
classes = ['normal', 'kitle', 'kalsifikasyon']

# ImageDataGenerator kullanarak veri ön işleme
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2  # %80 eğitim, %20 doğrulama için
)

# Eğitim verileri
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Doğrulama verileri
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Modeli tanımlayın
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # 3 sınıf: normal, kitle, kalsifikasyon
])

# Modeli derleyin
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

train_data_size = len(train_generator)

# Önerilen batch_size değeri
recommended_batch_size = 32

# batch_size değerini ayarlayın
batch_size = min(recommended_batch_size, train_data_size)

steps_per_epoch = train_data_size // batch_size

# Modeli eğitin
history = model.fit(
  train_generator,
    steps_per_epoch=train_data_size // batch_size,
    epochs=10,
    validation_data=validation_generator,  # Doğrulama verisini belirtin
    validation_steps=validation_data // batch_size

)

# Modeli kaydedin
model.save('/content/drive/MyDrive/Tekno/mammogram_cnn_model.h5')

# Eğitimin tarihçesini görselleştirin
plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
plt.xlabel('Epoch')
plt.ylabel('Doğruluk')
plt.legend(loc='lower right')
plt.show()

plt.plot(history.history['loss'], label='Eğitim Kaybı')
plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
plt.xlabel('Epoch')
plt.ylabel('Kaybı')
plt.legend(loc='upper right')
plt.show()

Found 82 images belonging to 7 classes.
Found 19 images belonging to 7 classes.


NameError: name 'validation_data' is not defined

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Veri yolunu ayarlayın
data_dir = output_folder  # PNG dosyalarının bulunduğu yol
classes = ['normal', 'kitle', 'kalsifikasyon']

# ImageDataGenerator kullanarak veri ön işleme
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2  # %80 eğitim, %20 doğrulama için
)

# Eğitim verileri
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Doğrulama verileri
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Modeli tanımlayın
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # 3 sınıf: normal, kitle, kalsifikasyon
])

# Modeli derle
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Veri artırma ve ön işleme
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(validation_data_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')
# Modeli kaydedin
model.save('/content/drive/MyDrive/Tekno/11.06-derleme.h5')

# Eğitimin tarihçesini görselleştirin
plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
plt.xlabel('Epoch')
plt.ylabel('Doğruluk')
plt.legend()
plt.show()


plt.plot(history.history['loss'], label='Eğitim Kaybı')
plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
plt.xlabel('Epoch')
plt.ylabel('Kaybı')
plt.legend()
plt.show()

Found 82 images belonging to 7 classes.
Found 19 images belonging to 7 classes.


NameError: name 'train_data_dir' is not defined

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Veri yolu
train_data_dir = '/content/drive/MyDrive/Tekno'
validation_data_dir = '/content/drive/MyDrive/Tekno'

# Modeli tanımla
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

# Modeli derle
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Veri artırma ve ön işleme
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

# Veri yükleyici
train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(validation_data_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')

# Modeli eğit
history = model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=10, validation_data=validation_generator, validation_steps=len(validation_generator))

# Eğitim ve doğrulama doğruluklarını görselleştir
plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
plt.xlabel('Epoch')
plt.ylabel('Doğruluk')
plt.legend()
plt.show()

Found 11 images belonging to 6 classes.
Found 11 images belonging to 6 classes.
